# 摘要任务
参考文章：[huggingface nlp learn summarization](https://huggingface.co/learn/nlp-course/chapter7/5?fw=pt#summarization)
## 任务目标:
训练一个双语文本摘要模型(英语、西班牙)
## 数据集准备
Multilingual Amazon Reviews Corpus （amazon 不在提供从其他途径下载到google drive 本地读取）  
该语料库由六种语言的亚马逊产品评论组成，通常用于对多语言分类器进行基准测试


In [ ]:
!pip install sentencepiece
!pip install transformers
!pip install datasets
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
Looking in indexes: https://download.pytorch.org/whl/cu118


In [ ]:
from datasets import load_dataset
sumDataset = load_dataset('csv', data_files={'train': '/content/drive/MyDrive/MultilingualAmazonReviews/test.csv', 'test': '/content/drive/MyDrive/MultilingualAmazonReviews/validation.csv'})
sumDataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 30000
    })
    test: Dataset({
        features: ['Unnamed: 0', 'review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 30000
    })
})

In [ ]:
enSumTrainDataset = sumDataset['train'].filter(lambda example: example['language'].startswith('en'))
print(enSumTrainDataset)
enSumTestDataset = sumDataset['test'].filter(lambda example: example['language'].startswith('en'))
print(enSumTestDataset)
esSumTrainDataset = sumDataset['train'].filter(lambda example: example['language'].startswith('es'))
print(esSumTrainDataset)
esSumTestDataset = sumDataset['test'].filter(lambda example: example['language'].startswith('es'))
print(esSumTestDataset)

Dataset({
    features: ['Unnamed: 0', 'review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
    num_rows: 5000
})
Dataset({
    features: ['Unnamed: 0', 'review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
    num_rows: 5000
})
Dataset({
    features: ['Unnamed: 0', 'review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
    num_rows: 5000
})
Dataset({
    features: ['Unnamed: 0', 'review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
    num_rows: 5000
})


In [ ]:
smTrainDataset = sumDataset['train'].filter(lambda example: example['language'].startswith('en') or example['language'].startswith('es')).shuffle(seed=42)
print(smTrainDataset)
smTestDataset = sumDataset['test'].filter(lambda example: example['language'].startswith('en') or example['language'].startswith('es')).shuffle(seed=42)
print(smTestDataset)

Dataset({
    features: ['Unnamed: 0', 'review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
    num_rows: 10000
})
Dataset({
    features: ['Unnamed: 0', 'review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
    num_rows: 10000
})


查看数据

In [ ]:
def show_samples(dataset, num_samples=3, seed=42):
    sample = dataset["train"].shuffle(seed=seed).select(range(num_samples))
    for example in sample:
        print(f"\n'>> Title: {example['review_title']}'")
        print(f"'>> Review: {example['review_body']}'")


show_samples(sumDataset)


'>> Title: Como dicen otras opiniones, la foto del producto es engañosa.'
'>> Review: La madera que aparece en la foto, parece una madera barnizada y bien tratada. La que llega en el paquete es una mala imitación mal tintada, con partes descoloridas donde se ha aplicado mal el tinte. El producto viene desmontado y sin instrucciones de montaje, cuando le pedimos al vendedor que nos envíe un pdf por correo electrónico para montarlo correctamente, nos contesta que ya nos lo enviará y a las 24 horas, manda una mala foto con una flecha (que adjunto), indicando que intentemos montarlo por favor. En resumen, el producto no es como el de la foto, y la atención postventa del vendedor es inexistente. No recomiendo la compra.'

'>> Title: Zurfrieden'
'>> Review: Funktioniert einwandfrei.Preis ok.Einfache Waage ohne schnick schnack aber genau das wollte ich.Würde die Waage wieder kaufen oder auch weiter empfehlen'

'>> Title: Calidad y eficacia con precio ajustado.'
'>> Review: Efectivo cumple bi

查看评论商品种类信息

In [ ]:
enSumTrainDataset.set_format("pandas")
english_df = enSumTrainDataset[:]
# Show counts for top 20 products
english_df["product_category"].value_counts()[:20]

home                      440
apparel                   390
wireless                  374
other                     324
beauty                    301
drugstore                 289
kitchen                   254
toy                       241
sports                    217
automotive                187
home_improvement          183
digital_ebook_purchase    173
lawn_and_garden           173
pet_products              168
pc                        155
electronics               153
office_product            139
shoes                     129
grocery                   115
book                      105
Name: product_category, dtype: int64

模型选择，对于多语言可以使用 mT5、mBART-50
### 编码器加载

In [ ]:
from transformers import AutoTokenizer
# seq2seq tokenizer need sentencepiece pip install it
tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")

You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
tokenizer.pad_token_id

0

In [ ]:
inputs = tokenizer("I loved reading the Hunger Games!")
inputs

{'input_ids': [336, 259, 28387, 11807, 287, 62893, 295, 12507, 309, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tokenizer.convert_ids_to_tokens(inputs.input_ids)

['▁I', '▁', 'loved', '▁reading', '▁the', '▁Hung', 'er', '▁Games', '!', '</s>']

In [ ]:
max_input_length = 512
max_target_length = 30


def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["review_body"],
        max_length=max_input_length,
        truncation=True,
    )
    labels = tokenizer(
        examples["review_title"], max_length=max_target_length, truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = smTrainDataset.map(preprocess_function, batched=True)
tokenized_datasets

Dataset({
    features: ['Unnamed: 0', 'review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 10000
})

In [ ]:
tokenized_datasets1 = smTestDataset.map(preprocess_function, batched=True)
tokenized_datasets1

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset({
    features: ['Unnamed: 0', 'review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 10000
})

指标评测  
ROUGE-L 计算最长公共子序列  
ROUGE-N 预测句子按N拆字计算召回率  
[参考](https://zhuanlan.zhihu.com/p/504279252)

In [ ]:
!pip install rouge_score
!pip install evaluate

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=e37e87a1a77702c473409be6a5f3fd6069a36a1adcd9c41b51dc3574c86df5c8
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 1.5 MB/s eta 0:00:00


In [ ]:
import evaluate
rouge_score = evaluate.load("rouge")

In [ ]:
print(rouge_score.compute.__doc__)

Compute the evaluation module.

        Usage of positional arguments is not allowed to prevent mistakes.

        Args:
            predictions (list/array/tensor, optional): Predictions.
            references (list/array/tensor, optional): References.
            **kwargs (optional): Keyword arguments that will be forwarded to the evaluation module :meth:`_compute`
                method (see details in the docstring).

        Return:
            dict or None

            - Dictionary with the results if this evaluation module is run on the main process (``process_id == 0``).
            - None if the evaluation module is not run on the main process (``process_id != 0``).
        
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tok

In [ ]:
generated_summary = "I absolutely loved reading the Hunger Games"
reference_summary = "I loved reading the Hunger Games"
scores = rouge_score.compute(
    predictions=[generated_summary], references=[reference_summary]
)
scores

{'rouge1': 0.923076923076923,
 'rouge2': 0.7272727272727272,
 'rougeL': 0.923076923076923,
 'rougeLsum': 0.923076923076923}

In [ ]:
scores["rouge1"]

0.923076923076923

In [ ]:
from transformers import pipeline
summarizer = pipeline("summarization",model="google/mt5-small")
articale = tokenized_datasets[1]['review_body']
summarizer(articale, max_length = 30, min_length = 5, do_sample=False)

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


[{'summary_text': '<extra_id_0>.) <extra_id_37>.'}]

模型

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-small")

定义 训练 args


In [ ]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.4 MB/s eta 0:00:00


In [ ]:
from transformers import Seq2SeqTrainingArguments

batch_size = 15
num_train_epochs = 8
# Show the training loss with every epoch
logging_steps = len(tokenized_datasets) // batch_size
model_name = "mt5-small"

args = Seq2SeqTrainingArguments(
    output_dir=f"{model_name}-finetuned-amazon-en-es",
    evaluation_strategy="epoch",
    learning_rate=5.6e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=num_train_epochs,
    predict_with_generate=True,
    logging_steps=logging_steps,
)

In [ ]:
!pip install nltk

In [ ]:
import nltk

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from nltk.tokenize import sent_tokenize


def three_sentence_summary(text):
    return "\n".join(sent_tokenize(text)[:3])

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Decode generated summaries into text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them
    # labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Decode reference summaries into text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # ROUGE expects a newline after each sentence
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    # Compute ROUGE scores
    result = rouge_score.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract the median scores
    result = {key: value.fmeasure * 100 for key, value in result.items()}
    return {k: v for k, v in result.items()}

官方文档测试函数

In [ ]:
import numpy as np

In [ ]:
def postprocess_text(preds, labels):
  preds = [pred.strip() for pred in preds]
  labels = [label.strip() for label in labels]

  # rougeLSum expects newline after each sentence
  preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
  labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

  return preds, labels

In [ ]:

def compute_metrics(eval_preds):
  preds, labels = eval_preds
  if isinstance(preds, tuple):
      preds = preds[0]
  # Replace -100s used for padding as we can't decode them
  preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
  decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  # Some simple post-processing
  decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

  result = rouge_score.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
  result = {k: round(v * 100, 4) for k, v in result.items()}
  prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
  result["gen_len"] = np.mean(prediction_lens)
  return result

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(
    enSumTrainDataset.column_names
)
tokenized_datasets

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 10000
})

In [ ]:
tokenized_datasets1 = tokenized_datasets1.remove_columns(
    enSumTrainDataset.column_names
)
tokenized_datasets1

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 10000
})

生成 decoder_input_ids 给 decoder 模型使用

In [ ]:
features = [tokenized_datasets[i] for i in range(2)]
data_collator(features)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': tensor([[  4295,  11569,  11765,    269,   1694,    263,    262,   3674,    375,
           1065,    800,    259,  16679,    260,   4295,    259,   9017,    843,
         201592,    263,    260,   1517, 120538,   3371,    361,   7387,    289,
          31510,    921,   4358,    269,    658,    738,   9109,    260,      1],
        [  4042,   2197,    261,    655,  10937,    299,    262,    261,    375,
           8601,  29782,    269,  20283,    870,    260,    653,    259,  15571,
            450,    259,    280,    651,  41275,    269,  37568,    260,    443,
           1113,    450,  10351,      1,      0,      0,      0,      0,      0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]]), 'labels': tensor([[  259, 74978,   674,  1057,  8924,   260,

In [ ]:
features1 = [tokenized_datasets1[i] for i in range(2)]
data_collator(features1)

{'input_ids': tensor([[   259,  74978,  24476,  29139,    435,    259,  42755,   5289,   6057,
            426,    283,  85724,    435,  59891,  13363,    319,    303,    289,
          68709,   1225,    260,   4691,   1094,   3861,   2197,    260,    501,
            259,  10987,    262, 111507,    362,  17025,    260,      1],
        [   303,  54719,    262,    259,    262,    707,    319,  23508,    316,
           4497,      1,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'labels': tensor([[ 1927,  4226,   263, 71399,  1928,   259,   276,   450, 60276, 

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets1,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,3.355800,3.268320,12.375800,5.091300,12.112800,12.139400,6.617800


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,3.355800,3.268320,12.375800,5.091300,12.112800,12.139400,6.617800
2,3.757800,3.049185,12.616400,5.489700,12.360600,12.368500,7.078100
3,3.523300,3.016106,12.751100,5.557000,12.514200,12.526700,6.812900
4,3.392100,2.985302,12.982800,5.779000,12.743800,12.766400,7.098600
5,3.303600,2.975738,13.178000,5.769500,12.940700,12.947700,7.149200
6,3.243600,2.966601,13.420900,6.034600,13.143100,13.167600,7.498500
7,3.215400,2.962067,13.301900,5.983400,13.035700,13.044900,7.360500
8,3.182300,2.963765,13.444100,6.039400,13.172500,13.183600,7.508900


TrainOutput(global_step=5336, training_loss=3.371467008404825, metrics={'train_runtime': 4295.2618, 'train_samples_per_second': 18.625, 'train_steps_per_second': 1.242, 'total_flos': 1.226322342144e+16, 'train_loss': 3.371467008404825, 'epoch': 8.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 2.9637651443481445,
 'eval_rouge1': 13.4441,
 'eval_rouge2': 6.0394,
 'eval_rougeL': 13.1725,
 'eval_rougeLsum': 13.1836,
 'eval_gen_len': 7.5089,
 'eval_runtime': 386.3548,
 'eval_samples_per_second': 25.883,
 'eval_steps_per_second': 1.726,
 'epoch': 8.0}

In [ ]:
# articale = tokenizer.decode(tokenized_datasets[2]['input_ids'])

'Worst iPhone charger ever. The charger head broke after used for 5 Times. Terrible quality.</s>'

In [ ]:
from transformers import pipeline
summarizer = pipeline("summarization",model="/content/mt5-small-finetuned-amazon-en-es/checkpoint-5000")
articale = "Worst iPhone charger ever. The charger head broke after used for 5 Times. Terrible quality."
summarizer(articale, max_length = 30, min_length = 5, do_sample=False)

Your max_length is set to 30, but your input_length is only 23. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=11)


[{'summary_text': 'Great iPhone charger ever.'}]